<a href="https://colab.research.google.com/github/eunjoo-ny/ML-Code/blob/main/10_PyTorch_Lab_10_2_mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
import torch.nn.init


In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
   torch.cuda.manual_seed_all(777)

In [38]:
#parameters
learning_rate= 0.001
training_epochs =15
batch_size = 100

In [39]:
#minst dataset
mnist_train =dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [40]:
#dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [75]:
#CNN Model (2 conv layers)
class CNN(torch.nn.Module):
      def __init__(self):
          super(CNN,self).__init__()
          self.layer1 = torch.nn.Sequential(#(,28,28,1)
              torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),#(,28,28,32)
              torch.nn.ReLU(),
              torch.nn.MaxPool2d(2)#(,14,14,32)
          )
          self.layer2 = torch.nn.Sequential(
              torch.nn.Conv2d(32,64,kernel_size=3, stride=1,padding=1),#(,14,14,64)
              torch.nn.ReLU(),
              torch.nn.MaxPool2d(2)
          )#(,7,7,64)

          self.layer3 = torch.nn.Sequential(
              torch.nn.Conv2d(64,128,kernel_size=3, stride=1,padding=1),#(,7,7,128)
              torch.nn.ReLU(),
              torch.nn.MaxPool2d(2)#(3,3,128)
          )
          
          
          self.fc1= torch.nn.Linear(3*3*128, 625, bias= True)
          


          
          self.relu=nn.ReLU()
              
          
          self.fc2 =torch.nn.Linear(625,10,bias=True)
          torch.nn.init.xavier_uniform_(self.fc1.weight)
          torch.nn.init.xavier_uniform_(self.fc2.weight)

          
          
          
      def forward(self,x):
          out = self.layer1(x)
          out = self.layer2(out)
          out = self.layer3(out)
          out = out.view(out.size(0),-1)
          out = self.fc1(out)
          out = self.relu(out)
          out = self.fc2(out)
          return out

In [76]:
model = CNN().to(device)
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)


In [77]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [78]:
total_batch= len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost=0

    for X,Y in data_loader:
        X= X.to(device)
        Y= Y.to(device)

        hypothesis=model(X)
        cost=criterion( hypothesis,Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost +=cost/total_batch

    print('[Epoch:{:>4}] cost ={:>.9}'.format(epoch+1, avg_cost))
     #print('[Epoch: {:.4d}] cost = {:.6f}'.format(epoch + 1, avg_cost))

print("Learning Finished")

Learning started. It takes sometime.
[Epoch:   1] cost =0.155675545
[Epoch:   2] cost =0.0431155562
[Epoch:   3] cost =0.0293267854
[Epoch:   4] cost =0.023795953
[Epoch:   5] cost =0.0180430189
[Epoch:   6] cost =0.0144161498
[Epoch:   7] cost =0.013050965
[Epoch:   8] cost =0.0121234953
[Epoch:   9] cost =0.00984117575
[Epoch:  10] cost =0.00816260651
[Epoch:  11] cost =0.00912692398
[Epoch:  12] cost =0.00558040477
[Epoch:  13] cost =0.0069900481
[Epoch:  14] cost =0.00440998096
[Epoch:  15] cost =0.00787348207
Learning Finished


In [79]:
with torch.no_grad():
     X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
     Y_test = mnist_test.test_labels.to(device)

     prediction= model(X_test)
     correct_prediction = torch.argmax(prediction, 1) ==Y_test
     accuracy = correct_prediction.float().mean()
     print('Accuracy:',accuracy.item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9879999756813049
